In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import
from __future__ import unicode_literals  

import os
import math
import numpy as np
import pandas as pd
import sklearn.linear_model as linear_model

import scipy
import sklearn

import influence.experiments as experiments
from influence.nlprocessor import NLProcessor
from influence.binaryLogisticRegressionWithLBFGS import BinaryLogisticRegressionWithLBFGS
from load_spam import load_spam

import tensorflow as tf

os.chdir('/Users/mr54725/Documents/repos/INF/')
np.random.seed(42)


Using TensorFlow backend.


In [2]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'


In [3]:
data_sets = load_spam()

In [4]:


num_classes = 2

input_dim = data_sets.train.x.shape[1]
weight_decay = 0.0001
# weight_decay = 1000 / len(lr_data_sets.train.labels)
batch_size = 100
initial_learning_rate = 0.001 
keep_probs = None
decay_epochs = [1000, 10000]
max_lbfgs_iter = 1000

tf.reset_default_graph()


In [5]:
tf_model = BinaryLogisticRegressionWithLBFGS(
    input_dim=input_dim,
    weight_decay=weight_decay,
    max_lbfgs_iter=max_lbfgs_iter,
    num_classes=num_classes, 
    batch_size=batch_size,
    data_sets=data_sets,
    initial_learning_rate=initial_learning_rate,
    keep_probs=keep_probs,
    decay_epochs=decay_epochs,
    mini_batch=False,
    train_dir='output',
    log_dir='log',
    model_name='spam_logreg')

tf_model.train()


Total number of parameters: 5338
Using normal model


/Users/mr54725/anaconda/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


LBFGS training took [58] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.026312243
Train loss (w/o reg) on all data: 0.012582187
Test loss (w/o reg) on all data: 0.23130246
Train acc on all data:  0.9992748368382887
Test acc on all data:   0.9468599033816425
Norm of the mean of gradients: 3.6910953e-07
Norm of the params: 16.571096


In [6]:
X_train = np.copy(tf_model.data_sets.train.x)
Y_train = np.copy(tf_model.data_sets.train.labels)
X_test = np.copy(tf_model.data_sets.test.x)
Y_test = np.copy(tf_model.data_sets.test.labels) 




In [7]:
num_train_examples = Y_train.shape[0] 
num_flip_vals = 6
num_check_vals = 6
num_random_seeds = 40

dims = (num_flip_vals, num_check_vals, num_random_seeds, 3)
fixed_influence_loo_results = np.zeros(dims)
fixed_loss_results = np.zeros(dims)
fixed_random_results = np.zeros(dims)

flipped_results = np.zeros((num_flip_vals, num_random_seeds, 3))



In [8]:
orig_results = tf_model.sess.run(
    [tf_model.loss_no_reg, tf_model.accuracy_op], 
    feed_dict=tf_model.all_test_feed_dict)


In [ ]:
print('Orig loss: %.5f. Accuracy: %.3f' % (orig_results[0], orig_results[1]))

for flips_idx in range(num_flip_vals):
    for random_seed_idx in range(num_random_seeds):
        
        random_seed = flips_idx * (num_random_seeds * 3) + (random_seed_idx * 2)        
        np.random.seed(random_seed)
    
        num_flips = int(num_train_examples / 20) * (flips_idx + 1)    
        idx_to_flip = np.random.choice(num_train_examples, size=num_flips, replace=False)
        Y_train_flipped = np.copy(Y_train)
        Y_train_flipped[idx_to_flip] = 1 - Y_train[idx_to_flip] 
        
        tf_model.update_train_x_y(X_train, Y_train_flipped)
        tf_model.train()        
        flipped_results[flips_idx, random_seed_idx, 1:] = tf_model.sess.run(
            [tf_model.loss_no_reg, tf_model.accuracy_op], 
            feed_dict=tf_model.all_test_feed_dict)
        print('Flipped loss: %.5f. Accuracy: %.3f' % (
                flipped_results[flips_idx, random_seed_idx, 1], flipped_results[flips_idx, random_seed_idx, 2]))
        
        train_losses = tf_model.sess.run(tf_model.indiv_loss_no_reg, feed_dict=tf_model.all_train_feed_dict)
        train_loo_influences = tf_model.get_loo_influences()

        for checks_idx in range(num_check_vals):
            np.random.seed(random_seed + 1)
            num_checks = int(num_train_examples / 20) * (checks_idx + 1)

            print('### Flips: %s, rs: %s, checks: %s' % (num_flips, random_seed_idx, num_checks))

            fixed_influence_loo_results[flips_idx, checks_idx, random_seed_idx, :], \
              fixed_loss_results[flips_idx, checks_idx, random_seed_idx, :], \
              fixed_random_results[flips_idx, checks_idx, random_seed_idx, :] \
              = experiments.test_mislabeled_detection_batch(
                tf_model, 
                X_train, Y_train,
                Y_train_flipped,
                X_test, Y_test, 
                train_losses, train_loo_influences,
                num_flips, num_checks)


Orig loss: 0.23130. Accuracy: 0.947
Using normal model
LBFGS training took [292] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.09586017
Train loss (w/o reg) on all data: 0.06288983
Test loss (w/o reg) on all data: 0.68472266
Train acc on all data:  0.9871887841430989
Test acc on all data:   0.8859903381642512
Norm of the mean of gradients: 1.050105e-05
Norm of the params: 25.678919
Flipped loss: 0.68472. Accuracy: 0.886
### Flips: 206, rs: 0, checks: 206
Using normal model
LBFGS training took [265] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.047704037
Train loss (w/o reg) on all data: 0.0292461
Test loss (w/o reg) on all data: 0.4380329
Train acc on all data:  0.9961324631375392
Test acc on all data:   0.9033816425120773
Norm of the mean of gradients: 1.3596321e-05
Norm of the params: 19.213503
     Influence (LOO): fixed 137 labels. Loss 0.43803. Accuracy 0.903.
Using normal model
LBFGS training took [247] iter.
After training with LBFGS: 

LBFGS training took [335] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.1006
Train loss (w/o reg) on all data: 0.067647725
Test loss (w/o reg) on all data: 0.4609756
Train acc on all data:  0.9859801788735799
Test acc on all data:   0.8888888888888888
Norm of the mean of gradients: 5.9097115e-06
Norm of the params: 25.67188
Flipped loss: 0.46098. Accuracy: 0.889
### Flips: 206, rs: 1, checks: 206
Using normal model
LBFGS training took [146] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.051550373
Train loss (w/o reg) on all data: 0.033563662
Test loss (w/o reg) on all data: 0.3041288
Train acc on all data:  0.9937152525985014
Test acc on all data:   0.9265700483091788
Norm of the mean of gradients: 3.9909755e-06
Norm of the params: 18.96666
     Influence (LOO): fixed 136 labels. Loss 0.30413. Accuracy 0.927.
Using normal model
LBFGS training took [203] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.04381682
Train loss (w/

LBFGS training took [248] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.09634335
Train loss (w/o reg) on all data: 0.06318563
Test loss (w/o reg) on all data: 0.6646311
Train acc on all data:  0.9886391104665216
Test acc on all data:   0.8347826086956521
Norm of the mean of gradients: 2.9230016e-05
Norm of the params: 25.751791
Flipped loss: 0.66463. Accuracy: 0.835
### Flips: 206, rs: 2, checks: 206
Using normal model
LBFGS training took [83] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.045698404
Train loss (w/o reg) on all data: 0.029091261
Test loss (w/o reg) on all data: 0.2675844
Train acc on all data:  0.9944404157602127
Test acc on all data:   0.9429951690821256
Norm of the mean of gradients: 8.247417e-07
Norm of the params: 18.22479
     Influence (LOO): fixed 147 labels. Loss 0.26758. Accuracy 0.943.
Using normal model
LBFGS training took [210] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.047422707
Train loss 

LBFGS training took [358] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.103110366
Train loss (w/o reg) on all data: 0.07178589
Test loss (w/o reg) on all data: 0.45149013
Train acc on all data:  0.9857384578196761
Test acc on all data:   0.9043478260869565
Norm of the mean of gradients: 9.181642e-06
Norm of the params: 25.029772
Flipped loss: 0.45149. Accuracy: 0.904
### Flips: 206, rs: 3, checks: 206
Using normal model
LBFGS training took [86] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.052776873
Train loss (w/o reg) on all data: 0.037053186
Test loss (w/o reg) on all data: 0.28429523
Train acc on all data:  0.9915397631133672
Test acc on all data:   0.9400966183574879
Norm of the mean of gradients: 7.217853e-07
Norm of the params: 17.733406
     Influence (LOO): fixed 141 labels. Loss 0.28430. Accuracy 0.940.
Using normal model
LBFGS training took [264] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.045513213
Train lo

LBFGS training took [237] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.101434976
Train loss (w/o reg) on all data: 0.06869209
Test loss (w/o reg) on all data: 0.4564158
Train acc on all data:  0.9845298525501571
Test acc on all data:   0.8917874396135266
Norm of the mean of gradients: 3.8492594e-06
Norm of the params: 25.590195
Flipped loss: 0.45642. Accuracy: 0.892
### Flips: 206, rs: 4, checks: 206
Using normal model
LBFGS training took [94] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.04908371
Train loss (w/o reg) on all data: 0.031389955
Test loss (w/o reg) on all data: 0.42432097
Train acc on all data:  0.9949238578680203
Test acc on all data:   0.9304347826086956
Norm of the mean of gradients: 8.4701395e-07
Norm of the params: 18.811571
     Influence (LOO): fixed 140 labels. Loss 0.42432. Accuracy 0.930.
Using normal model
LBFGS training took [141] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.047416523
Train lo

...

LBFGS training took [339] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.095991634
Train loss (w/o reg) on all data: 0.064387836
Test loss (w/o reg) on all data: 0.398705
Train acc on all data:  0.9879139473048103
Test acc on all data:   0.893719806763285
Norm of the mean of gradients: 7.69986e-06
Norm of the params: 25.14112
Flipped loss: 0.39871. Accuracy: 0.894
### Flips: 206, rs: 6, checks: 206
Using normal model
LBFGS training took [164] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.05362428
Train loss (w/o reg) on all data: 0.03515107
Test loss (w/o reg) on all data: 0.30551088
Train acc on all data:  0.9932318104906938
Test acc on all data:   0.9323671497584541
Norm of the mean of gradients: 2.073345e-06
Norm of the params: 19.22145
     Influence (LOO): fixed 135 labels. Loss 0.30551. Accuracy 0.932.
Using normal model
LBFGS training took [253] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.04191898
Train loss (w/o

LBFGS training took [275] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.09371601
Train loss (w/o reg) on all data: 0.062422898
Test loss (w/o reg) on all data: 0.5044837
Train acc on all data:  0.9871887841430989
Test acc on all data:   0.8908212560386474
Norm of the mean of gradients: 3.5728838e-06
Norm of the params: 25.017237
Flipped loss: 0.50448. Accuracy: 0.891
### Flips: 206, rs: 7, checks: 206
Using normal model
LBFGS training took [127] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.046387814
Train loss (w/o reg) on all data: 0.028511904
Test loss (w/o reg) on all data: 0.33122632
Train acc on all data:  0.9951655789219241
Test acc on all data:   0.9246376811594202
Norm of the mean of gradients: 1.895214e-06
Norm of the params: 18.908152
     Influence (LOO): fixed 137 labels. Loss 0.33123. Accuracy 0.925.
Using normal model
LBFGS training took [202] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.047757752
Train l

LBFGS training took [331] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.10125911
Train loss (w/o reg) on all data: 0.06871382
Test loss (w/o reg) on all data: 0.4055952
Train acc on all data:  0.9857384578196761
Test acc on all data:   0.9178743961352657
Norm of the mean of gradients: 6.0930747e-06
Norm of the params: 25.51286
Flipped loss: 0.40560. Accuracy: 0.918
### Flips: 206, rs: 8, checks: 206
Using normal model
LBFGS training took [139] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.052802756
Train loss (w/o reg) on all data: 0.03453162
Test loss (w/o reg) on all data: 0.3205517
Train acc on all data:  0.9939569736524051
Test acc on all data:   0.9371980676328503
Norm of the mean of gradients: 4.630209e-06
Norm of the params: 19.116037
     Influence (LOO): fixed 131 labels. Loss 0.32055. Accuracy 0.937.
Using normal model
LBFGS training took [249] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.048354283
Train loss 

LBFGS training took [274] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.091401234
Train loss (w/o reg) on all data: 0.06021926
Test loss (w/o reg) on all data: 0.4287567
Train acc on all data:  0.9874305051970027
Test acc on all data:   0.8801932367149758
Norm of the mean of gradients: 9.130379e-06
Norm of the params: 24.97278
Flipped loss: 0.42876. Accuracy: 0.880
### Flips: 206, rs: 9, checks: 206
Using normal model
LBFGS training took [75] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.048527867
Train loss (w/o reg) on all data: 0.031607393
Test loss (w/o reg) on all data: 0.2831903
Train acc on all data:  0.994198694706309
Test acc on all data:   0.9429951690821256
Norm of the mean of gradients: 1.3907706e-06
Norm of the params: 18.395908
     Influence (LOO): fixed 137 labels. Loss 0.28319. Accuracy 0.943.
Using normal model
LBFGS training took [190] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.044840794
Train loss 

LBFGS training took [267] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.097167164
Train loss (w/o reg) on all data: 0.065400995
Test loss (w/o reg) on all data: 0.5140105
Train acc on all data:  0.9871887841430989
Test acc on all data:   0.8888888888888888
Norm of the mean of gradients: 4.6448513e-06
Norm of the params: 25.205627
Flipped loss: 0.51401. Accuracy: 0.889
### Flips: 206, rs: 10, checks: 206
Using normal model
LBFGS training took [143] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.04950361
Train loss (w/o reg) on all data: 0.031063959
Test loss (w/o reg) on all data: 0.36025295
Train acc on all data:  0.9946821368141165
Test acc on all data:   0.9294685990338164
Norm of the mean of gradients: 4.336577e-06
Norm of the params: 19.203987
     Influence (LOO): fixed 136 labels. Loss 0.36025. Accuracy 0.929.
Using normal model
LBFGS training took [228] iter.
After training with LBFGS: 
Train loss (w reg) on all data: 0.047787286
Train 

In [8]:
flips_idx = 0
random_seed_idx=0